In [4]:
import pandas as pd
import requests

def geocode_city_state(city_state):
    url = f"https://nominatim.openstreetmap.org/search?format=json&q={city_state}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data:
            return data[0]['lat'], data[0]['lon']
    return None, None

# Read the original CSV file
original_csv_file = 'cleaned_cham_api_1900_1963.csv'
data = pd.read_csv(original_csv_file)

# Function to extract latitude and longitude from place
def extract_coordinates(row):
    place = row['place']
    longitude, latitude = geocode_city_state(place)
    return longitude, latitude

# Apply the function to the DataFrame
data['reclong'], data['reclat'] = zip(*data.apply(extract_coordinates, axis=1))

# Save the new CSV file
new_csv_file = 'cham_geo-dataset-v2.csv'
data.to_csv(new_csv_file, index=False)

print("New CSV file has been created with latitude and longitude columns.")


New CSV file has been created with latitude and longitude columns.


In [5]:
import pandas as pd
import json

def csv_to_geojson(csv_file):
    data = pd.read_csv(csv_file)
    
    features = []
    for _, row in data.iterrows():
        # Check if location coordinates are not NaN
        if not pd.isna(row['reclong']) and not pd.isna(row['reclat']):
            feature = {
                'type': 'Feature',
                'properties': {
                    'date': row['date'],
                    'title': row['title'],
                    'id': row['id'],
                    'place': row['place'],
                },
                'geometry': {
                    'type': 'Point',
                    'coordinates': [row['reclong'], row['reclat']]
                }
            }
            features.append(feature)
    
    geojson_data = {
        'type': 'FeatureCollection',
        'features': features
    }
    
    return json.dumps(geojson_data)

# Example usage:
csv_file = 'cham_geo-dataset.csv'
geojson = csv_to_geojson(csv_file)

# Save the GeoJSON to a file
output_geojson_file = 'cham-1900-1963-v3.geojson'
with open(output_geojson_file, 'w') as f:
    f.write(geojson)

print("success")


success
